1. **Video Preprocessing:**

    Use a video processing library like OpenCV to read the video frames, split the video into individual frames, and save them as images. You can specify a frame rate to control how many frames are extracted from the video.

In [16]:
import cv2

video_path = '/content/cctv_footage.mp4'
output_folder = '/content/frame_folder/'

cap = cv2.VideoCapture(video_path)
frame_rate = 15  # Adjust as needed
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if frame_count % frame_rate == 0:
        frame_filename = f"{output_folder}/frame_{frame_count}.jpg"
        cv2.imwrite(frame_filename, frame)

    frame_count += 1

cap.release()
cv2.destroyAllWindows()


In [17]:
output_folder

'/content/frame_folder/'

**2. Data Loading:**

    Use PyTorch's DataLoader to load and augment the data. You can use libraries like torchvision for common data augmentations.

In [30]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os

class FrameDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.frame_list = [f for f in os.listdir(root_dir) if f.endswith('.jpg')]

    def __len__(self):
        return len(self.frame_list)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.frame_list[idx])
        image = Image.open(img_name)

        if self.transform:
            image = self.transform(image)

        return image

# Update the path to your frame directory
frame_folder = '/content/frame_folder'
transform = transforms.Compose([
    transforms.Resize((128, 64)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

dataset = FrameDataset(root_dir=frame_folder, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [32]:
dataset

**3. Model Architecture:**

    Design the Siamese network or triplet network architecture. Here's an example of a Siamese network using ResNet as the base:

In [56]:
#modify
class SiameseNetwork(nn.Module):
    def __init__(self, base_model):
        super(SiameseNetwork, self).__init__()
        self.base_model = nn.Sequential(*list(base_model.children())[:-1])
        self.fc = nn.Sequential(
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
        )

    def forward_one(self, x):
        x = self.base_model(x)
        x = x.view(x.size()[0], -1)
        x = self.fc(x)
        return x  # Return a single tensor for the embedding

    def forward(self, input1, input2):
        output1 = self.forward_one(input1)
        output2 = self.forward_one(input2)
        return output1, output2  # Return both output1 and output2


**4. Loss Function:**

In [34]:
import torch

class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative):
        distance_positive = torch.norm(anchor - positive, p=2, dim=1)
        distance_negative = torch.norm(anchor - negative, p=2, dim=1)
        loss = torch.relu(distance_positive - distance_negative + self.margin)
        return loss.mean()


In [37]:
from torch.utils.data import DataLoader, Dataset
import random

class TripletDataset(Dataset):
    def __init__(self, frame_dataset):
        self.frame_dataset = frame_dataset

    def __len__(self):
        return len(self.frame_dataset)

    def __getitem__(self, idx):
        anchor = self.frame_dataset[idx]

        # Find a positive sample (another image of the same person)
        same_person_samples = [i for i in range(len(self.frame_dataset)) if i != idx]
        positive_idx = random.choice(same_person_samples)
        positive = self.frame_dataset[positive_idx]

        # Find a negative sample (an image of a different person)
        different_person_samples = [i for i in range(len(self.frame_dataset))]
        different_person_samples.remove(idx)  # Remove anchor index
        negative_idx = random.choice(different_person_samples)
        negative = self.frame_dataset[negative_idx]

        return anchor, positive, negative

# Update the path to your frame directory
frame_folder = '/content/frame_folder'
transform = transforms.Compose([
    transforms.Resize((128, 64)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

frame_dataset = FrameDataset(root_dir=frame_folder, transform=transform)
triplet_dataset = TripletDataset(frame_dataset)
dataloader = DataLoader(triplet_dataset, batch_size=32, shuffle=True)


**5. Training:**

    Implement the training loop. You will need to define an optimizer and iterate through your dataset.

In [58]:
import torch

device = torch.device("cpu")


In [59]:
def forward(self, input1, input2):
    output1 = self.forward_one(input1)
    output2 = self.forward_one(input2)
    return output1, output2

In [60]:
import torch.optim as optim

model = SiameseNetwork(models.resnet18(pretrained=True))
triplet_loss = TripletLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    for batch in dataloader:
        anchor, positive, negative = batch
        anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)

        optimizer.zero_grad()
        output1, output2 = model(anchor, positive)
        anchor_embedding = model(anchor, anchor)  # Compute the anchor's embedding separately
        loss = triplet_loss(output1, output2, anchor_embedding)
        loss.backward()
        optimizer.step()




/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


TypeError: ignored

**4. Evaluation:**

    After training, you can perform person re-identification on the video frames using the trained model. Here's an example of how you can extract features and compare them:

In [61]:
model.eval()
query_frame = cv2.imread('path_to_query_frame.jpg')
gallery_frames = [cv2.imread(f) for f in gallery_frame_filenames]

# Preprocess frames and convert to PyTorch tensors
transform = transforms.Compose([transforms.Resize((128, 64)), transforms.ToTensor()])
query_tensor = transform(query_frame).unsqueeze(0)  # Add batch dimension
gallery_tensors = [transform(frame) for frame in gallery_frames]

# Extract features
query_features = model.forward_one(query_tensor.to(device))
gallery_features = [model.forward_one(frame.to(device)) for frame in gallery_tensors]


similarity_scores = [torch.nn.functional.cosine_similarity(query_features, gf) for gf in gallery_features]

# Identify the index of the best match
best_match_index = torch.argmax(similarity_scores)


NameError: ignored